# Sentiment Analysis with Word2Vec

This notebook implements sentiment analysis on the [Multiclass Sentiment Analysis Dataset](https://huggingface.co/datasets/Sp1786/multiclass-sentiment-analysis-dataset) using three different approaches:

1. **Word2Vec (Pre-trained) + Multinomial Logistic Regression**
2. **Word2Vec (Pre-trained) + Multilayer Perceptron (MLP)**
3. **Standalone Multilayer Perceptron (Neural Network)**

## Table of Contents
1. Import Libraries & Load Data
2. Exploratory Data Analysis (EDA)
3. Data Preprocessing
4. Pipeline 1: Word2Vec + Logistic Regression
5. Pipeline 2: Word2Vec + MLP
6. Pipeline 3: Standalone MLP
7. Model Comparison

## 1. Import Libraries & Load Data

In [ ]:
# Install required packages (uncomment if needed)
# !pip install datasets gensim nltk scikit-learn tensorflow matplotlib seaborn wordcloud

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# NLP libraries
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import string

# Word2Vec
import gensim.downloader as api

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

# PyTorch
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

# Dataset
from datasets import load_dataset

# Utilities
from tqdm.auto import tqdm
from IPython.display import display

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

print("All libraries imported successfully!")

In [ ]:
# Load the dataset from Hugging Face
dataset = load_dataset("Sp1786/multiclass-sentiment-analysis-dataset")
print("Dataset loaded successfully!")
print(f"\nDataset structure: {dataset}")
print(f"\nSplits available: {list(dataset.keys())}")

In [ ]:
# Convert to pandas DataFrames for easier manipulation
df_train = pd.DataFrame(dataset['train'])
df_validation = pd.DataFrame(dataset['validation'])
df_test = pd.DataFrame(dataset['test'])

print(f"Training set shape: {df_train.shape}")
print(f"Validation set shape: {df_validation.shape}")
print(f"Test set shape: {df_test.shape}")

# Combine all data for EDA (we'll split again later)
df = pd.concat([df_train, df_validation, df_test], ignore_index=True)
print(f"\nTotal dataset shape: {df.shape}")

## 2. Exploratory Data Analysis (EDA)

In [ ]:
# Basic dataset info
print("Dataset Overview:")
print("="*50)
df.info()
print("\n")
df.head(10)

In [ ]:
# Check for missing values
print("Missing Values:")
print("="*50)
print(df.isnull().sum())
print(f"\nTotal missing values: {df.isnull().sum().sum()}")

In [ ]:
# Check for duplicates
print("Duplicate Analysis:")
print("="*50)
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")
print(f"Percentage of duplicates: {(duplicates/len(df))*100:.2f}%")

In [ ]:
# Sentiment label distribution
print("Sentiment Distribution:")
print("="*50)
print(df['sentiment'].value_counts())
print(f"\nUnique sentiments: {df['sentiment'].unique()}")

In [ ]:
# Visualize sentiment distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot
sentiment_counts = df['sentiment'].value_counts()
colors = ['#2ecc71', '#3498db', '#e74c3c']  # green, blue, red for positive, neutral, negative
axes[0].bar(sentiment_counts.index, sentiment_counts.values, color=colors)
axes[0].set_title('Sentiment Distribution (Count)', fontsize=14)
axes[0].set_xlabel('Sentiment')
axes[0].set_ylabel('Count')
for i, v in enumerate(sentiment_counts.values):
    axes[0].text(i, v + 200, str(v), ha='center', fontsize=11)

# Pie chart
axes[1].pie(sentiment_counts.values, labels=sentiment_counts.index, autopct='%1.1f%%', 
            colors=colors, explode=(0.02, 0.02, 0.02), startangle=90)
axes[1].set_title('Sentiment Distribution (Percentage)', fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
# Text length analysis
df['text_length'] = df['text'].apply(len)
df['word_count'] = df['text'].apply(lambda x: len(str(x).split()))

print("Text Statistics:")
print("="*50)
print(df[['text_length', 'word_count']].describe())

In [ ]:
# Visualize text length distribution by sentiment
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Text length distribution
for sentiment in df['sentiment'].unique():
    subset = df[df['sentiment'] == sentiment]
    axes[0].hist(subset['text_length'], bins=50, alpha=0.6, label=sentiment)
axes[0].set_title('Text Length Distribution by Sentiment', fontsize=14)
axes[0].set_xlabel('Character Count')
axes[0].set_ylabel('Frequency')
axes[0].legend()
axes[0].set_xlim(0, 500)  # Focus on majority of data

# Word count distribution
for sentiment in df['sentiment'].unique():
    subset = df[df['sentiment'] == sentiment]
    axes[1].hist(subset['word_count'], bins=50, alpha=0.6, label=sentiment)
axes[1].set_title('Word Count Distribution by Sentiment', fontsize=14)
axes[1].set_xlabel('Word Count')
axes[1].set_ylabel('Frequency')
axes[1].legend()
axes[1].set_xlim(0, 100)  # Focus on majority of data

plt.tight_layout()
plt.show()

In [ ]:
# Box plot of text statistics by sentiment
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

df.boxplot(column='text_length', by='sentiment', ax=axes[0])
axes[0].set_title('Text Length by Sentiment', fontsize=14)
axes[0].set_xlabel('Sentiment')
axes[0].set_ylabel('Character Count')
axes[0].set_ylim(0, 500)

df.boxplot(column='word_count', by='sentiment', ax=axes[1])
axes[1].set_title('Word Count by Sentiment', fontsize=14)
axes[1].set_xlabel('Sentiment')
axes[1].set_ylabel('Word Count')
axes[1].set_ylim(0, 100)

plt.suptitle('')  # Remove automatic title
plt.tight_layout()
plt.show()

In [ ]:
# Sample texts from each sentiment class
print("Sample Texts by Sentiment:")
print("="*80)
for sentiment in df['sentiment'].unique():
    print(f"\n{sentiment.upper()}:")
    print("-"*40)
    samples = df[df['sentiment'] == sentiment]['text'].head(3).values
    for i, sample in enumerate(samples, 1):
        print(f"{i}. {sample[:150]}..." if len(sample) > 150 else f"{i}. {sample}")

## 3. Data Preprocessing

In [ ]:
# Text preprocessing function
def preprocess_text(text):
    """
    Preprocess text for NLP tasks:
    - Convert to lowercase
    - Remove URLs
    - Remove mentions (@username)
    - Remove hashtags
    - Remove special characters and numbers
    - Remove extra whitespaces
    - Tokenize
    - Remove stopwords
    - Lemmatize
    """
    # Convert to lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove mentions and hashtags
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Tokenize
    tokens = word_tokenize(text)
    
    # Remove stopwords and lemmatize
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    
    tokens = [lemmatizer.lemmatize(token) for token in tokens 
              if token not in stop_words and len(token) > 2]
    
    return ' '.join(tokens)

# Test preprocessing
sample_text = "I absolutely LOVE this product!!! 😍 Check it out: https://example.com @user123 #amazing"
print("Original:", sample_text)
print("Preprocessed:", preprocess_text(sample_text))

In [ ]:
# Apply preprocessing to the entire dataset
print("Preprocessing texts... This may take a few minutes.")
df['cleaned_text'] = df['text'].apply(preprocess_text)
print("Preprocessing completed!")

# Show before and after
print("\nBefore and After Preprocessing:")
print("="*80)
for i in range(3):
    print(f"\nOriginal: {df['text'].iloc[i][:100]}...")
    print(f"Cleaned:  {df['cleaned_text'].iloc[i][:100]}...")

In [ ]:
# Remove empty texts after preprocessing
initial_size = len(df)
df = df[df['cleaned_text'].str.len() > 0]
final_size = len(df)
print(f"Removed {initial_size - final_size} empty texts after preprocessing")
print(f"Final dataset size: {final_size}")

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['sentiment'])

print("Label Encoding:")
print("="*50)
for i, label in enumerate(label_encoder.classes_):
    print(f"{label} -> {i}")

In [ ]:
# Train-validation-test split
X = df['cleaned_text'].values
y = df['label'].values

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"Train size: {len(X_train)}")
print(f"Validation size: {len(X_val)}")
print(f"Test size: {len(X_test)}")

## 4. Pipeline 1: Word2Vec + Logistic Regression

In [ ]:
# Load pre-trained Word2Vec model (may take a while on first run)
print("Loading pre-trained Word2Vec model...")
w2v_model = api.load('word2vec-google-news-300')
embedding_dim = w2v_model.vector_size
print(f"Word2Vec model loaded with embedding dimension: {embedding_dim}")


def document_vector(text):
    tokens = text.split()
    vectors = [w2v_model[word] for word in tokens if word in w2v_model]
    if not vectors:
        return np.zeros(embedding_dim)
    return np.mean(vectors, axis=0)


# Vectorize datasets
print("Vectorizing datasets with Word2Vec embeddings...")
X_train_w2v = np.vstack([document_vector(text) for text in X_train])
X_val_w2v = np.vstack([document_vector(text) for text in X_val])
X_test_w2v = np.vstack([document_vector(text) for text in X_test])

print("Word2Vec embeddings ready!")

In [ ]:
# Train multinomial logistic regression on Word2Vec embeddings
log_reg = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
log_reg.fit(X_train_w2v, y_train)

# Evaluate
val_preds_lr = log_reg.predict(X_val_w2v)
val_proba_lr = log_reg.predict_proba(X_val_w2v)

print("Validation Accuracy (LogReg + Word2Vec):", accuracy_score(y_val, val_preds_lr))
print("\nClassification Report:\n", classification_report(y_val, val_preds_lr, target_names=label_encoder.classes_))

In [ ]:
# Confusion matrix for Logistic Regression
cm_lr = confusion_matrix(y_val, val_preds_lr)
plt.figure(figsize=(6,5))
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix: Word2Vec + Logistic Regression')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

## 5. Pipeline 2: Word2Vec + PyTorch MLP

In [ ]:
class Word2VecDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


class Word2VecMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, dropout=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, len(label_encoder.classes_))
        )

    def forward(self, x):
        return self.net(x)


def train_model(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == targets).sum().item()
        total += targets.size(0)
    return running_loss / total, correct / total


def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            running_loss += loss.item() * inputs.size(0)
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            correct += (preds == targets).sum().item()
            total += targets.size(0)
    return running_loss / total, correct / total, np.array(all_preds)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

train_dataset_w2v = Word2VecDataset(X_train_w2v, y_train)
val_dataset_w2v = Word2VecDataset(X_val_w2v, y_val)
test_dataset_w2v = Word2VecDataset(X_test_w2v, y_test)

train_loader_w2v = DataLoader(train_dataset_w2v, batch_size=64, shuffle=True)
val_loader_w2v = DataLoader(val_dataset_w2v, batch_size=128, shuffle=False)
test_loader_w2v = DataLoader(test_dataset_w2v, batch_size=128, shuffle=False)

model_w2v_mlp = Word2VecMLP(input_dim=embedding_dim, hidden_dim=256, dropout=0.3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_w2v_mlp.parameters(), lr=1e-3)

best_val_acc = 0.0
patience = 3
patience_counter = 0
best_state = None

for epoch in range(15):
    train_loss, train_acc = train_model(model_w2v_mlp, train_loader_w2v, criterion, optimizer, device)
    val_loss, val_acc, val_preds_w2v = evaluate_model(model_w2v_mlp, val_loader_w2v, criterion, device)

    print(f"Epoch {epoch+1:02d} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        best_state = model_w2v_mlp.state_dict()
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

if best_state is not None:
    model_w2v_mlp.load_state_dict(best_state)

print(f"Best validation accuracy: {best_val_acc:.4f}")

In [ ]:
val_loss_mlp, val_acc_mlp, val_preds_mlp = evaluate_model(model_w2v_mlp, val_loader_w2v, criterion, device)
print(f"Validation Accuracy (Word2Vec + PyTorch MLP): {val_acc_mlp:.4f}")
print("\nClassification Report:\n", classification_report(y_val, val_preds_mlp, target_names=label_encoder.classes_))

cm_mlp = confusion_matrix(y_val, val_preds_mlp)
plt.figure(figsize=(6,5))
sns.heatmap(cm_mlp, annot=True, fmt='d', cmap='Purples', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix: Word2Vec + PyTorch MLP')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

## 6. Pipeline 3: TF-IDF + PyTorch MLP

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print("TF-IDF matrices prepared")

In [ ]:
class SparseTfidfDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        x_dense = self.X[idx].toarray().squeeze()
        return torch.tensor(x_dense, dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.long)


class TfidfMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, dropout=0.4):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, len(label_encoder.classes_))
        )

    def forward(self, x):
        return self.net(x)


In [ ]:
train_dataset_tfidf = SparseTfidfDataset(X_train_tfidf, y_train)
val_dataset_tfidf = SparseTfidfDataset(X_val_tfidf, y_val)
test_dataset_tfidf = SparseTfidfDataset(X_test_tfidf, y_test)

train_loader_tfidf = DataLoader(train_dataset_tfidf, batch_size=64, shuffle=True)
val_loader_tfidf = DataLoader(val_dataset_tfidf, batch_size=128, shuffle=False)
test_loader_tfidf = DataLoader(test_dataset_tfidf, batch_size=128, shuffle=False)

model_tfidf = TfidfMLP(input_dim=X_train_tfidf.shape[1], hidden_dim=512, dropout=0.4).to(device)
criterion_tfidf = nn.CrossEntropyLoss()
optimizer_tfidf = optim.Adam(model_tfidf.parameters(), lr=1e-3)

best_val_acc_tfidf = 0.0
patience = 3
patience_counter = 0
best_state_tfidf = None

for epoch in range(20):
    train_loss, train_acc = train_model(model_tfidf, train_loader_tfidf, criterion_tfidf, optimizer_tfidf, device)
    val_loss, val_acc, val_preds_tfidf = evaluate_model(model_tfidf, val_loader_tfidf, criterion_tfidf, device)

    print(f"Epoch {epoch+1:02d} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_val_acc_tfidf:
        best_val_acc_tfidf = val_acc
        patience_counter = 0
        best_state_tfidf = model_tfidf.state_dict()
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

if best_state_tfidf is not None:
    model_tfidf.load_state_dict(best_state_tfidf)

print(f"Best validation accuracy: {best_val_acc_tfidf:.4f}")

In [ ]:
val_loss_tf, val_acc_tf, val_preds_tf = evaluate_model(model_tfidf, val_loader_tfidf, criterion_tfidf, device)
print(f"Validation Accuracy (TF-IDF + PyTorch MLP): {val_acc_tf:.4f}")
print("\nClassification Report:\n", classification_report(y_val, val_preds_tf, target_names=label_encoder.classes_))

cm_tf = confusion_matrix(y_val, val_preds_tf)
plt.figure(figsize=(6,5))
sns.heatmap(cm_tf, annot=True, fmt='d', cmap='Greens', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix: TF-IDF + PyTorch MLP')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

## 7. Test Evaluation & Model Comparison

In [ ]:
test_preds_lr = log_reg.predict(X_test_w2v)
test_acc_lr = accuracy_score(y_test, test_preds_lr)

_, test_acc_mlp, test_preds_mlp = evaluate_model(model_w2v_mlp, test_loader_w2v, criterion, device)
_, test_acc_tf, test_preds_tf = evaluate_model(model_tfidf, test_loader_tfidf, criterion_tfidf, device)

comparison_df = pd.DataFrame({
    'Pipeline': [
        'Word2Vec + Logistic Regression',
        'Word2Vec + PyTorch MLP',
        'TF-IDF + PyTorch MLP'
    ],
    'Validation Accuracy': [
        accuracy_score(y_val, val_preds_lr),
        val_acc_mlp,
        val_acc_tf
    ],
    'Test Accuracy': [
        test_acc_lr,
        test_acc_mlp,
        test_acc_tf
    ]
})

print("Validation & Test Accuracy Comparison:")
display(comparison_df.sort_values('Test Accuracy', ascending=False))

In [ ]:
print("Test Classification Report - Word2Vec + Logistic Regression:\n", classification_report(y_test, test_preds_lr, target_names=label_encoder.classes_))
print("Test Classification Report - Word2Vec + PyTorch MLP:\n", classification_report(y_test, test_preds_mlp, target_names=label_encoder.classes_))
print("Test Classification Report - TF-IDF + PyTorch MLP:\n", classification_report(y_test, test_preds_tf, target_names=label_encoder.classes_))

## 8. Key Takeaways
- Word2Vec features coupled with a PyTorch MLP provide a flexible nonlinear decision boundary.
- TF-IDF with a PyTorch MLP can capture nuanced n-gram patterns absent in averaged Word2Vec representations.
- Logistic regression remains a competitive baseline and is faster to train, but shows lower recall on minority classes.
- Further improvements could leverage contextual embeddings (e.g., transformers) or class-weighted losses to handle any imbalance.